# 将 lora 用在自定义模型上

In [1]:
from torch import nn


class MLP(nn.Module):
    def __init__(self, num_units_hidden=2000):
        super().__init__()
        self.seq = nn.Sequential(
            nn.Linear(20, num_units_hidden),
            nn.ReLU(),
            nn.Linear(num_units_hidden, num_units_hidden),
            nn.ReLU(),
            nn.Linear(num_units_hidden, 2),
            nn.LogSoftmax(dim=-1),
        )

    def forward(self, X):
        return self.seq(X)

In [3]:
from pprint import pprint
pprint([(n, type(m)) for n, m in MLP().named_modules()])

[('', <class '__main__.MLP'>),
 ('seq', <class 'torch.nn.modules.container.Sequential'>),
 ('seq.0', <class 'torch.nn.modules.linear.Linear'>),
 ('seq.1', <class 'torch.nn.modules.activation.ReLU'>),
 ('seq.2', <class 'torch.nn.modules.linear.Linear'>),
 ('seq.3', <class 'torch.nn.modules.activation.ReLU'>),
 ('seq.4', <class 'torch.nn.modules.linear.Linear'>),
 ('seq.5', <class 'torch.nn.modules.activation.LogSoftmax'>)]


假设我们要将 LoRA 应用于输入层和隐藏层，即 'seq.0' 和 'seq.2' 。此外，假设我们想要在没有 LoRA 的情况下更新输出层，那就是 'seq.4' 。相应的配置将是：

In [4]:
from peft import LoraConfig

config = LoraConfig(
    target_modules=["seq.0", "seq.2"],
    modules_to_save=["seq.4"],
)

c:\tech\Anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


bin c:\tech\Anaconda3\envs\nlp\lib\site-packages\bitsandbytes\libbitsandbytes_cuda121.dll


In [5]:
from peft import get_peft_model

model = MLP()
peft_model = get_peft_model(model, config)
peft_model.print_trainable_parameters()
# prints trainable params: 56,164 || all params: 4,100,164 || trainable%: 1.369798866581922

trainable params: 56,164 || all params: 4,100,164 || trainable%: 1.369798866581922
